# Cryptostock Analytica

In [91]:
# Initial imports
import os
import time
import requests
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import hvplot.pandas
import holoviews as hv
import numpy as np

%matplotlib inline

In [92]:
def fetch_stock_data(stock_symbol, start_date, end_date):
    stock_data = yf.download(stock_symbol, start=start_date, end=end_date)
    stock_data.reset_index(inplace=True)
    stock_data['Date'] = pd.to_datetime(stock_data['Date'], utc=True)
    return stock_data

# Ask user for the stock symbol
selected_stock = input("Enter the stock symbol you want to view data for: ").upper()

# Calculate the date range
end_date = pd.Timestamp.now(tz='UTC')
start_date = end_date - pd.DateOffset(years=3)

# Get the data for the selected stock
selected_stock_data = fetch_stock_data(selected_stock, start_date, end_date)

if not selected_stock_data.empty:
    # Round the price data to 2 decimal places
    selected_stock_data[['Open', 'High', 'Low', 'Close']] = selected_stock_data[['Open', 'High', 'Low', 'Close']].round(2)

    # Print the data
    print(selected_stock_data)
else:
    print("Error: Invalid stock symbol or no data available.")


selected_stock_data.set_index('Date')

[*********************100%***********************]  1 of 1 completed
                         Date    Open    High     Low   Close   Adj Close  \
0   2020-05-11 00:00:00+00:00   77.03   79.26   76.81   78.75   77.426300   
1   2020-05-12 00:00:00+00:00   79.46   79.92   77.73   77.85   76.541443   
2   2020-05-13 00:00:00+00:00   78.04   78.99   75.80   76.91   75.617271   
3   2020-05-14 00:00:00+00:00   76.13   77.45   75.38   77.39   76.081818   
4   2020-05-15 00:00:00+00:00   75.09   76.97   75.05   76.93   75.632011   
..                        ...     ...     ...     ...     ...         ...   
749 2023-05-02 00:00:00+00:00  170.09  170.35  167.54  168.54  168.539993   
750 2023-05-03 00:00:00+00:00  169.50  170.92  167.16  167.45  167.449997   
751 2023-05-04 00:00:00+00:00  164.89  167.04  164.31  165.79  165.789993   
752 2023-05-05 00:00:00+00:00  170.98  174.30  170.76  173.57  173.570007   
753 2023-05-08 00:00:00+00:00  172.48  173.85  172.11  173.50  173.500000   

      

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-05-11 00:00:00+00:00,77.03,79.26,76.81,78.75,77.426300,145946400
2020-05-12 00:00:00+00:00,79.46,79.92,77.73,77.85,76.541443,162301200
2020-05-13 00:00:00+00:00,78.04,78.99,75.80,76.91,75.617271,200622400
2020-05-14 00:00:00+00:00,76.13,77.45,75.38,77.39,76.081818,158929200
2020-05-15 00:00:00+00:00,75.09,76.97,75.05,76.93,75.632011,166348400
...,...,...,...,...,...,...
2023-05-02 00:00:00+00:00,170.09,170.35,167.54,168.54,168.539993,48425700
2023-05-03 00:00:00+00:00,169.50,170.92,167.16,167.45,167.449997,65136000
2023-05-04 00:00:00+00:00,164.89,167.04,164.31,165.79,165.789993,81235400


In [93]:
def fetch_top_n_cryptos(n=10):
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        'vs_currency': 'usd',
        'order': 'market_cap_desc',
        'per_page': n,
        'page': 1,
        'sparkline': False,
    }
    response = requests.get(url, params=params)
    data = response.json()
    return data

def fetch_crypto_data(coin_id, from_timestamp, to_timestamp):
    url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart/range"
    params = {
        'vs_currency': 'usd',
        'from': from_timestamp,
        'to': to_timestamp
    }
    response = requests.get(url, params=params)
    data = response.json()
    df = pd.DataFrame(data['prices'], columns=['time', 'price'])
    df['time'] = pd.to_datetime(df['time'], unit='ms')
    return df

def get_data_by_name(name):
    for crypto_id, data in crypto_data.items():
        if crypto_id.lower() == name.lower():
            return data
    raise ValueError(f"No data found for '{name}'")

# Fetch the top 10 cryptocurrencies by market cap
top_cryptos = fetch_top_n_cryptos(10)

# Unix timestamps for three years of data
to_timestamp = pd.Timestamp.now(tz='UTC').timestamp()
from_timestamp = pd.Timestamp.now(tz='UTC') - pd.DateOffset(years=3)
from_timestamp = from_timestamp.timestamp()

# Fetch historical data for each of the top 10 cryptos
crypto_data = {}
for crypto in top_cryptos:
    coin_id = crypto['id']
    try:
        df = fetch_crypto_data(coin_id, from_timestamp, to_timestamp)
        crypto_data[coin_id] = df
        print(f"Fetched data for {coin_id}")
        time.sleep(0.5)  # Add a delay between requests to avoid rate limiting
    except Exception as e:
        print(f"Error fetching data for {coin_id}: {e}")



Fetched data for bitcoin
Fetched data for ethereum
Fetched data for tether
Fetched data for binancecoin
Fetched data for usd-coin
Fetched data for ripple
Error fetching data for cardano: 'prices'
Error fetching data for staked-ether: 'prices'
Error fetching data for dogecoin: 'prices'
Error fetching data for matic-network: 'prices'


In [94]:
def get_data_by_name(name):
    for crypto in top_cryptos:
        if crypto['name'].lower() == name.lower():
            coin_id = crypto['id']
            return crypto_data[coin_id]
    return None

# Ask user for the name of the cryptocurrency
selected_crypto = input("Enter the name of the cryptocurrency you want to view data for: ")

# Get the data for the selected cryptocurrency
selected_crypto_data = get_data_by_name(selected_crypto)

if selected_crypto_data is not None:
    # Round the price data to 2 decimal places
    selected_crypto_data['price'] = selected_crypto_data['price'].round(2)

    # Print the data
    print(selected_crypto_data)
else:
    print("Error: Invalid cryptocurrency name.")


selected_crypto_data.tail()



           time     price
0    2020-05-10   9566.78
1    2020-05-11   8752.62
2    2020-05-12   8604.75
3    2020-05-13   8788.47
4    2020-05-14   9283.09
...         ...       ...
1090 2023-05-05  28846.46
1091 2023-05-06  29520.32
1092 2023-05-07  28887.74
1093 2023-05-08  28611.44
1094 2023-05-09  27696.76

[1095 rows x 2 columns]


,time,price
1090,2023-05-05,28846.46
1091,2023-05-06,29520.32
1092,2023-05-07,28887.74
1093,2023-05-08,28611.44
1094,2023-05-09,27696.76


In [95]:
# selected_crypto_data = selected_crypto_data.rename(columns= {'time' : 'Date'})
# selected_crypto_data


Creating a DF combing closing prices where the dates match

In [96]:
# Mergin the date frame on the date 
selected_stock_data['Date'] = selected_stock_data['Date'].dt.tz_convert(None)
combined_data = pd.merge(selected_crypto_data, selected_stock_data, left_on='time', right_on='Date', how='inner')
combined_data

,time,price,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-05-11,8752.62,2020-05-11,77.03,79.26,76.81,78.75,77.426300,145946400
1,2020-05-12,8604.75,2020-05-12,79.46,79.92,77.73,77.85,76.541443,162301200
2,2020-05-13,8788.47,2020-05-13,78.04,78.99,75.80,76.91,75.617271,200622400
3,2020-05-14,9283.09,2020-05-14,76.13,77.45,75.38,77.39,76.081818,158929200
4,2020-05-15,9796.49,2020-05-15,75.09,76.97,75.05,76.93,75.632011,166348400
...,...,...,...,...,...,...,...,...,...
749,2023-05-02,28125.50,2023-05-02,170.09,170.35,167.54,168.54,168.539993,48425700
750,2023-05-03,28654.39,2023-05-03,169.50,170.92,167.16,167.45,167.449997,65136000
751,2023-05-04,28988.32,2023-05-04,164.89,167.04,164.31,165.79,165.789993,81235400
752,2023-05-05,28846.46,2023-05-05,170.98,174.30,170.76,173.57,173.570007,113316400


In [97]:
# cleaning data by dropping values that we are not needed 

combined_data = combined_data.drop(columns= ['Open', 'time','Open', 'High', 'Low', 'Adj Close', 'Volume'])
combined_data

,price,Date,Close
0,8752.62,2020-05-11,78.75
1,8604.75,2020-05-12,77.85
2,8788.47,2020-05-13,76.91
3,9283.09,2020-05-14,77.39
4,9796.49,2020-05-15,76.93
...,...,...,...
749,28125.50,2023-05-02,168.54
750,28654.39,2023-05-03,167.45
751,28988.32,2023-05-04,165.79
752,28846.46,2023-05-05,173.57


In [98]:
# Final data frame with the closing prices and date as Index and reanaming columns with their corresponfing asset

combined_data.set_index('Date', inplace= True)
combined_data.rename(columns={'price': selected_crypto, 'Close': selected_stock}, inplace=True)

combined_data

,bitcoin,AAPL
Date,,
2020-05-11,8752.62,78.75
2020-05-12,8604.75,77.85
2020-05-13,8788.47,76.91
2020-05-14,9283.09,77.39
2020-05-15,9796.49,76.93
...,...,...
2023-05-02,28125.50,168.54
2023-05-03,28654.39,167.45
2023-05-04,28988.32,165.79


Creating a data frame with the percent change for the assets

In [99]:
Stock_cryoto_pct = combined_data.pct_change().dropna()
Stock_cryoto_pct

,bitcoin,AAPL
Date,,
2020-05-12,-0.016894,-0.011429
2020-05-13,0.021351,-0.012075
2020-05-14,0.056281,0.006241
2020-05-15,0.055305,-0.005944
2020-05-18,-0.013286,0.023528
...,...,...
2023-05-02,-0.042114,-0.006191
2023-05-03,0.018805,-0.006467
2023-05-04,0.011654,-0.009913


 **Correlation Analysis**:

In [100]:

# this a correlation function for the coosen assets
def pearson_correlation(crypto, stocks):
    return pd.Series(crypto).corr(pd.Series(stocks))


correlation = pearson_correlation(selected_crypto_data['price'],selected_stock_data['Close'])
print(correlation)



0.6628071518183503


In [101]:


# Create a scatter plot using Hvplot
scatter_plot = combined_data.hvplot.scatter(
    x=selected_crypto,
    y=selected_stock,
    title='Scatter Plot of Crypto Prices vs. Stock Prices'
)

x_position = 0.05 * combined_data[selected_crypto].max()  # Adjust the x position of the label
y_position = 0.95 * combined_data[selected_stock].max()  # Adjust the y position of the label
correlation_label = hv.Text((x_position + 10000), y_position, f'Correlation: {correlation:.2f}', fontsize=14)

# Combine scatter plot and correlation label
plot_with_label = scatter_plot * correlation_label

# Display the plot
plot_with_label.opts(legend_position='top_left', height=400, width=900)
                      


:Overlay
   .Scatter.I :Scatter   [bitcoin]   (AAPL)
   .Text.I    :Text   [x,y]

Create if statements to print a text in regards to the correlation of the assets 

In [102]:
def calculate_rsi_from_yahoo(ticker_symbol, n=14):
    ticker = yf.Ticker(ticker_symbol)
    hist_data = ticker.history(period="max")
    prices = hist_data['Close'].to_numpy()
    deltas = np.diff(prices)
    seed = deltas[:n+1]
    up = seed[seed >= 0].sum() / n
    down = -seed[seed < 0].sum() / n
    rs = up / down
    rsi = np.zeros_like(prices)
    rsi[:n] = 100. - 100. / (1. + rs)
    for i in range(n, len(prices)):
        delta = deltas[i - 1]
        if delta > 0:
            upval = delta
            downval = 0.
        else:
            upval = 0.
            downval = -delta
        up = (up * (n - 1) + upval) / n
        down = (down * (n - 1) + downval) / n
        rs = up / down
        rsi[i] = 100. - 100. / (1. + rs)
    rsi_df = hist_data.iloc[n:].copy()
    rsi_df["RSI"] = rsi[n:]
    return rsi_df
ticker_symbol = selected_stock
rsi_df = calculate_rsi_from_yahoo(ticker_symbol)
rsi_plot = rsi_df.hvplot.line(
    x='Date', y='RSI', title=f"Relative Strength Index (RSI) for {ticker_symbol}",
    xlabel="Date", ylabel="RSI", line_color="blue", width=800, height=400,
    ylim=(0, 100), shared_axes=False, yticks=[0, 30, 70, 100],
    hover_cols=['Open', 'High', 'Low', 'Close', 'Volume']
)
rsi_plot.opts(
    # title_fontsize=20, title_font='serif',
    # xlabel_fontsize=16, ylabel_fontsize=16,
    # legend_position='top_left', legend_fontsize=14,
    tools=['hover'], toolbar='above'
)


:Curve   [Date]   (RSI,Open,High,Low,Close,Volume)

 **Risk Analysis**:

***stadart devaition**

In [103]:

stock_and_crypto_returns_df = combined_data = pd.DataFrame({
    'Crypto Prices': selected_crypto_data['price'].pct_change().dropna(),
    'Stock Prices': selected_stock_data['Close'].pct_change().dropna()
})
crypto_std_annualized = np.sqrt(252) * stock_and_crypto_returns_df['Crypto Prices'].std()
stocks_std_annualized = np.sqrt(252) * stock_and_crypto_returns_df['Stock Prices'].std()

assets_annualized_std = pd.DataFrame({
    "Asset type" : ['Crypto' , 'Stock'],
   " annualized_std ": [crypto_std_annualized , stocks_std_annualized]})


assets_annualized_std.hvplot.bar( title= "Annualized Standart Deviation" , x = "Asset type" )



:Bars   [Asset type]   ( annualized_std )

*** Std rolling window**

We will use the rolling standart deviation to see the monthly volatility of the assets


In [118]:


Rolling_21_day_std = Stock_cryoto_pct.rolling(window= 21).std() * np.sqrt(21)

std_plot = Rolling_21_day_std.hvplot(
    title="Rolling 21-Day Standard Deviation",
    width=900,
    height=400,
    stacked=False,
    color=['blue', 'orange'],
   
    ylabel="Standard Deviation",

    ylim=(0, Rolling_21_day_std.max().max() * 1.1),
)

std_plot



:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [105]:
def calculate_bollinger_bands_from_yahoo(ticker_symbol, period='3y', window=20, sigma=2):
    # Download historical data from Yahoo Finance
    data = yf.download(ticker_symbol, period=period)
    # Calculate rolling mean and standard deviation
    rolling_mean = data['Close'].rolling(window=window).mean()
    rolling_std = data['Close'].rolling(window=window).std()
    # Calculate Upper and Lower Bands
    upper_band = rolling_mean + (sigma * rolling_std)
    lower_band = rolling_mean - (sigma * rolling_std)
    # Create a DataFrame with the Close, Upper Band, and Lower Band columns
    bollinger_bands = pd.DataFrame({
        'Close': data['Close'],
        'Upper Band': upper_band,
        'Lower Band': lower_band
    })
    # Drop rows with NaN values
    bollinger_bands.dropna(inplace=True)
    return bollinger_bands
ticker_symbol = selected_stock
bollinger_bands = calculate_bollinger_bands_from_yahoo(ticker_symbol)
# Create a DataFrame with the Close, Upper Band, and Lower Band columns
df = pd.DataFrame({
    'Close': bollinger_bands['Close'],
    'Upper Band': bollinger_bands['Upper Band'],
    'Lower Band': bollinger_bands['Lower Band']
})
# Plot the DataFrame using hvplot.line
plot = df.hvplot.line(
    title=f"Bollinger Bands for {ticker_symbol}",
    xlabel="Period",
    ylabel="Price",
    legend="top_left",
     height = 400,
    width = 1200
)
# Show the plot
display(plot)


[*********************100%***********************]  1 of 1 completed


:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [106]:
def plot_sma(ticker_symbol):
    # Retrieve the historical price data
    data = yf.download(ticker_symbol, period='3y')
    # Calculate the 50-day simple moving average
    sma = data['Close'].rolling(window=50).mean()
    # Plot the closing price and the simple moving average using hvplot
    return data.hvplot.line(x='Date', y='Close', ylabel='Price') * sma.hvplot.line(x='Date', y='Close', color='red')


plot_sma(selected_stock)




[*********************100%***********************]  1 of 1 completed


:Overlay
   .Curve.I     :Curve   [Date]   (Close)
   .Curve.Close :Curve   [Date]   (Close)

In [112]:
# Define function to download stock data and calculate historical volatility
def get_volatility(ticker, start_date, end_date, window_size):
    # Download stock data from Yahoo Finance
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    # Calculate historical volatility (standard deviation of daily returns)
    stock_data['Historical Volatility'] = stock_data['Adj Close'].pct_change().rolling(window=window_size).std() * np.sqrt(window_size)
    return stock_data
# Define function to get historical volatility of a sector ETF
def get_sector_volatility(sector_etf, start_date, end_date, window_size):
    # Download sector ETF data from Yahoo Finance
    sector_data = yf.download(sector_etf, start=start_date, end=end_date)
    # Calculate historical volatility (standard deviation of daily returns)
    sector_data['Historical Volatility'] = sector_data['Adj Close'].pct_change().rolling(window=window_size).std() * np.sqrt(window_size)
    return sector_data
# Define function to plot historical volatility of a stock and its sector
def plot_volatility_comparison(stock_data, sector_data, ticker, sector_etf):
    # Merge stock and sector data on date index
    data = stock_data[['Historical Volatility']].merge(sector_data[['Historical Volatility']], left_index=True, right_index=True, suffixes=(f' {ticker}', f' {sector_etf}'))
    # Create line charts of historical volatility for stock and sector
    chart = data.hvplot.line(
        x='Date', y=[f'Historical Volatility {ticker}', f'Historical Volatility {sector_etf}'],
        xlabel='Date', ylabel='Historical Volatility', title=f'Historical Volatility Comparison of {ticker} and {sector_etf}',
        width=800, height=400, legend='top',
    )
    return chart
# Example usage: Compare Apple (AAPL) to S & P 500 (SPY)
ticker = 'AAPL'
sector_etf = 'SPY'
start_date = '2020-05-08'
end_date = '2023-05-08'
window_size = 21
# Get historical volatility data for Apple and XLK
stock_data = get_volatility(ticker, start_date, end_date, window_size)
sector_data = get_sector_volatility(sector_etf, start_date, end_date, window_size)
# Plot historical volatility comparison
plot_volatility_comparison(stock_data, sector_data, ticker, sector_etf)




[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


:NdOverlay   [Variable]
   :Curve   [Date]   (value)